In [1]:
import numpy as np

import pybop

# Parameter set and model definition
parameter_set = pybop.ParameterSet.pybamm("Chen2020")
model = pybop.lithium_ion.SPM(parameter_set=parameter_set)

# Fitting parameters
parameters = [
    pybop.Parameter(
        "Negative electrode active material volume fraction",
        prior=pybop.GaussianLogPrior(0.68, 0.05),
    ),
    pybop.Parameter(
        "Positive electrode active material volume fraction",
        prior=pybop.GaussianLogPrior(0.58, 0.05),
    ),
]

# Generate data
init_soc = 0.5
sigma = 0.001
experiment = pybop.Experiment(
    [
        (
            "Discharge at 0.5C for 3 minutes (3 second period)",
            "Charge at 0.5C for 3 minutes (3 second period)",
        ),
    ]
    * 2
)
values = model.predict(init_soc=init_soc, experiment=experiment)


def noise(sigma):
    return np.random.normal(0, sigma, len(values["Voltage [V]"].data))


# Form dataset
dataset = pybop.Dataset(
    {
        "Time [s]": values["Time [s]"].data,
        "Current function [A]": values["Current [A]"].data,
        "Voltage [V]": values["Voltage [V]"].data + noise(sigma),
        "Bulk open-circuit voltage [V]": values["Bulk open-circuit voltage [V]"].data
        + noise(sigma),
    }
)

signal = ["Voltage [V]", "Bulk open-circuit voltage [V]"]


# Generate problem, cost function, and optimisation class
problem = pybop.FittingProblem(
    model, parameters, dataset, signal=signal, init_soc=init_soc
)
likelihood = pybop.GaussianLogLikelihoodKnownSigma(problem, sigma=[0.02, 0.02])
prior1 = pybop.GaussianLogPrior(0.6, 0.02)
prior2 = pybop.GaussianLogPrior(0.5, 0.02)
composed_prior = pybop.ComposedLogPrior(prior1, prior2)
posterior = pybop.LogPosterior(likelihood)  # , log_prior=prior)

In [5]:
composed_prior.evaluateS1([0.7, 0.6])

(-19.01383105555304, array([125., 125.]))

In [ ]:
prior = 0.0
for i, val in enumerate([0.6, 0.7]):
    prior += posterior._prior[i]([val])
print(prior)

-0.006412519301365105


In [ ]:
posterior([0.6, 0.7])
posterior.prior()

ValueError: Error in cost calculation: 'list' object is not callable

In [ ]:
prior._sigma2

AttributeError: 'numpy.float64' object has no attribute '_sigma2'

In [ ]:
prior._multip

array([-1250., -1250.])

In [ ]:
prior._offset

array([2.99308447, 2.99308447])

In [ ]:
prior.loc

array([0.6, 0.7])